In [ ]:
# Add all annotated brains to the viewer
from timeit import  default_timer as timer
import os, sys

import neuroglancer
import numpy as np
import pandas as pd
import ast
%load_ext autoreload
%autoreload 2

HOME = os.path.expanduser("~")
DIR = os.path.join(HOME, 'programming/pipeline_utility')

sys.path.append(DIR)
from utilities.atlas.utilities_contour import get_contours_from_annotations, add_structure_to_neuroglancer, \
    create_full_volume
from utilities.FileLocationManager import DATA_PATH
from utilities.SqlController import SqlController

In [ ]:
xy_ng_resolution_um = 5
color_radius = 3
animal = 'MD589'
sqlController = SqlController(animal)

x_length = 1000
y_length = 1000
z_length = 300

#y_voxels = 1+int( y_length*0.46*(.46/xy_ng_resolution_um) + 0.5)
#x_voxels = 1+int( x_length*0.46*(.46/xy_ng_resolution_um) + 0.5)
#full_brain_volume_annotated = np.zeros((z_length,y_voxels,x_voxels), dtype=np.uint8)
full_brain_volume_annotated = np.zeros((z_length,y_length,x_length), dtype=np.uint8)

In [ ]:
#neuroglancer.set_server_bind_address(bind_port='33645')
#viewer = neuroglancer.Viewer()
animal = 'MD589'

csvfile = os.path.join(DATA_PATH, 'atlas_data/foundation_brain_annotations',  f'{animal}_annotation.csv')

hand_annotations = pd.read_csv(csvfile)
hand_annotations['vertices'] = hand_annotations['vertices'] \
    .apply(lambda x: x.replace(' ', ',')) \
    .apply(lambda x: x.replace('\n', ',')) \
    .apply(lambda x: x.replace(',]', ']')) \
    .apply(lambda x: x.replace(',,', ',')) \
    .apply(lambda x: x.replace(',,', ',')) \
    .apply(lambda x: x.replace(',,', ',')).apply(lambda x: x.replace(',,', ','))

hand_annotations['vertices'] = hand_annotations['vertices'].apply(lambda x: ast.literal_eval(x))

structures = sqlController.get_structures_dict()

#
structures_arr = hand_annotations.name.unique()
annotation_structures = structures_arr.tolist()

In [ ]:
[a for a in annotation_structures if a in structures]

In [ ]:
#annotation_structures

In [ ]:
full_brain_volume_annotated = np.zeros((z_length,1500,3000), dtype=np.uint8)
print('Reset full brain volume shape:', full_brain_volume_annotated.shape)
for structure, values in structures.items():
    color = values[1]

    print(structure, color, end="\t")
    contour_annotations, first_sec, last_sec = get_contours_from_annotations(animal, structure, hand_annotations, densify=4)
    if first_sec == 0 or last_sec == 0:
        print('No sections found')
        continue
    else:
        print('Section start, end:' ,first_sec, last_sec, end="\t")
        

    threshold = 1
    structure_volume, xyz_offsets = create_full_volume(contour_annotations, structure, first_sec, last_sec, \
        color_radius, xy_ng_resolution_um, threshold, color)
    
    x_start, y_start, z_start = xyz_offsets
    x_end = x_start + structure_volume.shape[2]
    y_end = y_start + structure_volume.shape[1]
    z_end = z_start + structure_volume.shape[0]
    print('X range', x_start, x_end, end="\t")
    print('Y range', y_start, y_end, end="\t")
    print('Z range', z_start, z_end)
    try:
        full_brain_volume_annotated[z_start:z_end, y_start:y_end,x_start:x_end] += structure_volume
    except:
        print('Could not create')

In [ ]:
viewer = neuroglancer.Viewer()
viewer

In [ ]:
full_brain_volume_annotated = np.swapaxes(full_brain_volume_annotated, 0, 2)

In [ ]:
all_volume_layer = neuroglancer.SegmentationLayer(
    source = neuroglancer.LocalVolume(
        data=full_brain_volume_annotated, 
        dimensions=neuroglancer.CoordinateSpace(names=['z', 'y', 'x'], units='um', scales=[20, 5, 5]), 
        voxel_offset=(0, 0, 0)
    ),
)

with viewer.txn() as s:
    s.layers.clear()
    s.layers['all'] = all_volume_layer

In [ ]:

OUTPUT = os.path.join('/net/birdstore/Active_Atlas_Data/data_root/CSHL_volumes', animal)
outfile = os.path.join(OUTPUT, 'full_brain_volume_annotated.npy')
print('full_brain_volume_annotated at', OUTPUT)
np.save(outfile, np.ascontiguousarray(full_brain_volume_annotated))